##This notebook is for basic QC on sequencing data

In [3]:
# %install_ext https://raw.githubusercontent.com/SchlossLab/ipython-mothurmagic/master/mothurmagic.py
# Only needs to be done once - gets the mothurmagic so you can run mother easily in the %notebook

Installed mothurmagic.py. To use it, type:
  %load_ext mothurmagic


In [2]:
%load_ext mothurmagic
# Loads mothurmagic so we can run mothur in the notebook using %%mothur at the top of the cell

In [3]:
%%mothur
help()

mothur > help()
For more information about a specific command type 'commandName(help)' i.e. 'cluster(help)'

For further assistance please refer to the Mothur manual on our wiki at http://www.mothur.org/wiki, or contact Pat Schloss at mothur.bugs@gmail.com.

mothur > quit()


In [2]:
!echo $PATH
#Make sure mothur folder is in your path somehow. If not, enter 

/opt/virt_env/bin:/anaconda/bin:/Users/Thea/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/usr/texbin:/anaconda/bin/samtools-0.1.19/:/anaconda/bin/bowtie2-2.2.5/:/anaconda/bin/cufflinks-2.2.1.OSX_x86_64/:/anaconda/bin/tophat-2.1.0.OSX_x86_64/


In [3]:
!export PATH=$PATH:/opt/virt_env/bin/mothur

First we need to cut out sequences with errors

In [91]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-07-03.assembled.demult.fastq \
-fastaout ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering
# Note we needed to remove - dashes from the name for future mothur processing

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

01:28 1.6Mb  100.0% Filtering, 97.9% passed
   1916021  FASTQ recs (1.9M)              
     39782  Low qual recs discarded (expected errs > 1.00)
   1876239  Converted (1.9M, 97.9%)


####If the file were bigger, we would need to split it (below)

In [49]:
!wc -l ../../SeqData/pear_merged-2015-06-30.assembled.fastq
# Counts the number of lines in the fastq file. They come in groups of 4 lines.

 33057800 ../../SeqData/pear_merged-2015-06-30.assembled.fastq


In [28]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering - the whole file is too big to do at once so needs to be split

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu



usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.fastq -fastaout ../../SeqData/pear_merged-2015-06-30.assembled.maxee.fasta -fastq_maxee 1

---Fatal error---
File size too big for 32-bit version (6.0Gb)


In [53]:
!head -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq > \
../../SeqData/pear_merged-2015-06-30.assembled.split1.fastq
!tail -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq > \
../../SeqData/pear_merged-2015-06-30.assembled.split2.fastq
# Split the data for usearch into two files, divided by the number of lines it has as determined above

^C


### We're gonna run ls 
It seems to work

In [7]:
ls

Assign_taxonomy.ipynb      demultiplex.ipynb
CatchAllCmdL.exe           fasta_number.py
LICENSE                    merge_reads.ipynb
MapFile_corrected.txt      mothur.1435872215.logfile
OTU_binning.ipynb          mothur.1435877909.logfile
Playing_around.ipynb       mothur.1435961587.logfile
QC_basic.ipynb             mothur.1435967680.logfile
blast/                     uchime*


In [ ]:
!head -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq

In [51]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.split1.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.split1.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering on our split fastq file

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

02:04 1.6Mb  100.0% Filtering, 96.0% passed
   4132225  FASTQ recs (4.1M)              
    165212  Low qual recs discarded (expected errs > 1.00)
   3967013  Converted (4.0M, 96.0%)


In [62]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.split2.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.split2.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering on our split fastq file

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

02:07 1.6Mb  100.0% Filtering, 96.2% passed
   4132225  FASTQ recs (4.1M)              
    155057  Low qual recs discarded (expected errs > 1.00)
   3977168  Converted (4.0M, 96.2%)


In [63]:
!cat ../../SeqData/pear_merged-2015-06-30.assembled.split1.maxee.fasta \
../../SeqData/pear_merged-2015-06-30.assembled.split2.maxee.fasta > \
../../SeqData/pear_merged_2015_06_30.assembled.maxee.fasta
# Joining the files back together into one fasta file.

###Now we examine and cut the sequences using mothur

In [93]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta)

mothur > summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta)

Using 1 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	50	50	0	2	1
2.5%-tile:	1	297	297	0	4	46906
25%-tile:	1	299	299	0	4	469060
Median: 	1	299	299	0	5	938120
75%-tile:	1	299	299	0	5	1407180
97.5%-tile:	1	300	300	0	6	1829334
Maximum:	1	492	492	0	49	1876239
Mean:	1	299.449	299.449	0	4.69148
# of Seqs:	1876239

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.summary

It took 61 secs to summarize 1876239 sequences.

mothur > quit()


Okay, so we have about 2M sequences after demultiplexing and the maxee control. There are no ambiguous bases. Most are in the correct size range, actually.

I want to remove the PCR primers from the dataset. I will use trimmomatic here, not to do any quality filtering, etc., but just to cut those sequences out.

In [31]:
!cutadapt -g GTGCCAGCMGCCGCGGTAA  -o ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers.fasta ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta

This is cutadapt 1.8.1 with Python 2.7.9
Command line parameters: -g GTGCCAGCMGCCGCGGTAA -o ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers.fasta ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta
Trimming 1 adapter with at most 10.0% errors in single-end mode ...
Finished in 45.75 s (24 us/read; 2.46 M reads/minute).

=== Summary ===

Total reads processed:               1,876,239
Reads with adapters:                 1,835,780 (97.8%)
Reads written (passing filters):     1,876,239 (100.0%)

Total basepairs processed:   561,838,161 bp
Total written (filtered):    520,888,701 bp (92.7%)

=== Adapter 1 ===

Sequence: GTGCCAGCMGCCGCGGTAA; Type: regular 5'; Length: 19; Trimmed: 1835780 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1

Overview of removed sequences
length	count	expect	max.err	error counts
3	495	29316.2	0	495
4	64	7329.1	0	64
5	22	1832.3	0	22
6	8	458.1	0	8
7	18	114.5	0	18
8	18	28.6	0	18
9	24	7.2	0	24
10	134	1.8	1	107 27
11	211	0.4	1	13

In [32]:
!cutadapt -a GATTAGANACCCNDGTAGTCC   -o ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.fasta ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers.fasta

This is cutadapt 1.8.1 with Python 2.7.9
Command line parameters: -a GATTAGANACCCNDGTAGTCC -o ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.fasta ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers.fasta
Trimming 1 adapter with at most 10.0% errors in single-end mode ...
Finished in 85.75 s (46 us/read; 1.31 M reads/minute).

=== Summary ===

Total reads processed:               1,876,239
Reads with adapters:                 1,841,516 (98.1%)
Reads written (passing filters):     1,876,239 (100.0%)

Total basepairs processed:   520,888,701 bp
Total written (filtered):    475,614,017 bp (91.3%)

=== Adapter 1 ===

Sequence: GATTAGANACCCNDGTAGTCC; Type: regular 3'; Length: 21; Trimmed: 1841516 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20-21 bp: 2

Bases preceding removed adapters:
  A: 0.1%
  C: 0.0%
  G: 99.8%
  T: 0.1%
  none/other: 0.0%
    The adapter is preceded by "G" extremely often.
    The provided adapter sequence may be incomp

In [34]:
!head -200 ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.fasta

>67.2_0
GACGTAGGGTGCAAGCGTTGTCCGGATTTATTGGGCATAAAGAGCTCGTAGGCGGCTTGTTGCGTCGACCGTGAAAACCTACCGCTTAACGGTGGGCTTGCGGTCGATACGGGCAGGCTAGAGTTCGGTAGGGGAGACTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGGTCTCTGGGCCGATACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAG
>74.2_1
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGATCGATCAGTCAGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTTTGATACTGTCGATCTGGAGTATGGAAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAACACCAGTGGCGAAGGCGGCTCACTGGTCCATTACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAG
>67_2
TACAGAGGGTGCAAGCGTTGTTCGGAATTATTGGGCGTAAAGGGTGCGTAGGCGGTGCGGTAAGTCTTTTGTGAAATCTCCGGGCTCAACTCGGAGCCTGCAGAAGAAACTGCCGTGCTGGAGTATGGGAGAGGTGAGTGGAATTCCCGGTGTAGCGGTGAAATGCGTAGATATCGGGAGGAACACCTGTGGCGAAAGCGGCTCACTGGACCATCACTGACGCTGATGCACGAAAGCTAGGGGAGCAAACAG
>1_3
TACAGAGGTGGCAAGCGTTGTTCGGAATTACTGGGCGTAAAGGGCGCGTAGGCGGCCTTCTAAGTCAGACGTGAAATCCCCCGGCTTAACCTGGGAACTGCGTCTGATACTGGGAGGCTTGAGTATGGGAGAGGGATGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGATATCTGGAGGAATACCGGTGGCGAAGGCGGCATCCTGGACCAT

Looks good. We will proceed through standard mothur operating procedure. (First testing it with a cropped dataset)

In [28]:
!head -250000 ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.fasta > ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.crop.fasta

In [3]:
%%mothur
unique.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.fasta)

mothur > unique.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.fasta)
1876239	418270

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.names
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.fasta


mothur > quit()


6mers 63s
7mers 55s
8mers 56s, 53s (61s with larger pcr db, 68s with even larger, 60s with medium, 68s with current, 69s with filtered current)
9mers 52s, 54s
10mers 62s
8 and 9mers seem comparable - just use 8 because good AND standard.

For filtered wider PCR database,
6mers 81s
7mers 70s
8mers 69s
9mers 62s, 61s <- we shall use this one.
10mers 75s

In [4]:
%%mothur
align.seqs(candidate=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.fasta, template=../../SeqData/db/silva.total.filter.pcr.filter.fasta, processors=4, flip=T, ksize=9)

mothur > align.seqs(candidate=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.fasta, template=../../SeqData/db/silva.total.filter.pcr.filter.fasta, processors=4, flip=T, ksize=9)

Using 4 processors.

Reading in the ../../SeqData/db/silva.total.filter.pcr.filter.fasta template sequences...	DONE.
It took 2 to read  18491 sequences.
Aligning sequences from ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.fasta ...
Some of you sequences generated alignments that eliminated too many bases, a list is provided in ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.flip.accnos. If the reverse compliment proved to be better it was reported.
It took 403 secs to align 418270 sequences.


Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.align
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.align.report
../../SeqData/pear_merged_2015_07_03.assembl

^ Started at 3:52 PM finished at 4:12 PM; 11:05AM finished at 11:12 AM

In [5]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.names, processors=4)

mothur > summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.names, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	-1	-1	0	0	1	1
2.5%-tile:	987	3812	251	0	3	46906
25%-tile:	987	3812	252	0	4	469060
Median: 	987	3812	252	0	5	938120
75%-tile:	987	3812	252	0	5	1407180
97.5%-tile:	987	3812	253	0	6	1829334
Maximum:	4000	4000	417	0	49	1876239
Mean:	995.332	3793.4	249.417	0	4.65093
# of unique seqs:	418270
total # of seqs:	1876239

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.summary

It took 20 secs to summarize 1876239 sequences.

mothur > quit()


In [8]:
%%mothur
screen.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.names, start=987, end=3812, minlength=251, maxlength=255, maxhomop=8, processors=4)

mothur > screen.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.names, start=987, end=3812, minlength=251, maxlength=255, maxhomop=8, processors=4)

Using 4 processors.

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.align
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.bad.accnos
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names


It took 32 secs to screen 418270 sequences.

mothur > quit()


In [9]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names, processors=4)

mothur > summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	82	3812	251	0	3	1
2.5%-tile:	987	3812	252	0	4	45371
25%-tile:	987	3812	252	0	4	453710
Median: 	987	3812	252	0	5	907420
75%-tile:	987	3812	252	0	5	1361130
97.5%-tile:	987	3812	253	0	6	1769469
Maximum:	987	3813	255	0	8	1814839
Mean:	986.999	3812	252.028	0	4.67404
# of unique seqs:	361926
total # of seqs:	1814839

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.summary

It took 18 secs to summarize 1814839 sequences.

mothur > quit()


In [10]:
%%mothur
filter.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.align, vertical=T, trump=., processors=4)

mothur > filter.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.align, vertical=T, trump=., processors=4)

Using 4 processors.
Creating Filter...


Running Filter...



Length of filtered alignment: 506
Number of columns removed: 3494
Length of the original alignment: 4000
Number of sequences used to construct filter: 361926

Output File Names:
../../SeqData/pear_merged_2015_07_03.filter
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.fasta


mothur > quit()


This filter step make sure that the sequences are all comparable. Now they start around the same place, and will end around the same place, so they will be comparable.

In [11]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names, processors=4)

mothur > summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	504	232	0	3	1
2.5%-tile:	1	506	252	0	4	45371
25%-tile:	1	506	252	0	4	453710
Median: 	1	506	252	0	5	907420
75%-tile:	1	506	252	0	5	1361130
97.5%-tile:	1	506	253	0	6	1769469
Maximum:	1	506	255	0	8	1814839
Mean:	1	506	252.027	0	4.67403
# of unique seqs:	361926
total # of seqs:	1814839

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.summary

It took 5 secs to summarize 1814839 sequences.

mothur > quit()


In [14]:
%%mothur
screen.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names, minlength=251, maxlength=254, maxhomop=8, processors=4)

mothur > screen.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.names, minlength=251, maxlength=254, maxhomop=8, processors=4)

Using 4 processors.

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.fasta
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.bad.accnos
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.good.names


It took 6 secs to screen 361926 sequences.

mothur > quit()


In [15]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.good.names, processors=4)

mothur > summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.good.names, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	504	251	0	3	1
2.5%-tile:	1	506	252	0	4	45354
25%-tile:	1	506	252	0	4	453536
Median: 	1	506	252	0	5	907072
75%-tile:	1	506	252	0	5	1360607
97.5%-tile:	1	506	253	0	6	1768789
Maximum:	1	506	254	0	8	1814142
Mean:	1	506	252.026	0	4.67399
# of unique seqs:	361632
total # of seqs:	1814142

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.summary

It took 5 secs to summarize 1814142 sequences.

mothur > quit()


In [16]:
%%mothur
deunique.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.good.names)

mothur > deunique.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.good.good.names)

Output File Names:
../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.redundant.fasta


mothur > quit()


In [18]:
# FinalQC.fasta will be our last file.
# Here we are taking out all the alignment characters (-,.,etc.)
nprocs=4
!sed '/>/! s/-//g;/>/! s/\.//g' ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.primers2.unique.good.filter.good.redundant.fasta > ../../SeqData/16SfinalQC.fasta

In [19]:
!head -100 ../../SeqData/16SfinalQC.fasta

>67.2_0
GACGTAGGGTGCAAGCGTTGTCCGGATTTATTGGGCATAAAGAGCTCGTAGGCGGCTTGTTGCGTCGACCGTGAAAACCTACCGCTTAACGGTGGGCTTGCGGTCGATACGGGCAGGCTAGAGTTCGGTAGGGGAGACTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGGTCTCTGGGCCGATACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAG
>74.2_1
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGATCGATCAGTCAGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTTTGATACTGTCGATCTGGAGTATGGAAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAACACCAGTGGCGAAGGCGGCTCACTGGTCCATTACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAG
>70.2_140
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGATCGATCAGTCAGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTTTGATACTGTCGATCTGGAGTATGGAAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAACACCAGTGGCGAAGGCGGCTCACTGGTCCATTACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAG
>19_180
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGATCGATCAGTCAGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTTTGATACTGTCGATCTGGAGTATGGAAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAACACCAGTGGCGAAGGCGGCTCACT

In [20]:
%%mothur
summary.seqs(fasta=../../SeqData/16SfinalQC.fasta, processors=4)

mothur > summary.seqs(fasta=../../SeqData/16SfinalQC.fasta, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	251	251	0	3	1
2.5%-tile:	1	252	252	0	4	45354
25%-tile:	1	252	252	0	4	453536
Median: 	1	252	252	0	5	907072
75%-tile:	1	252	252	0	5	1360607
97.5%-tile:	1	253	253	0	6	1768789
Maximum:	1	254	254	0	8	1814142
Mean:	1	252.026	252.026	0	4.67399
# of Seqs:	1814142

Output File Names:
../../SeqData/16SfinalQC.summary

It took 13 secs to summarize 1814142 sequences.

mothur > quit()


###Here is where we made the databases from mothur's reference Silva database. We took eukaryotic, archaeal, and bacterial sequences that should have hit our primers, combine them, cut them to our range, and filter out gaps.

In [11]:
!mkdir ../../SeqData/db/
!curl -o ../../SeqData/db/silva_B.zip http://www.mothur.org/w/images/9/98/Silva.bacteria.zip
!curl -o ../../SeqData/db/silva_E.zip http://www.mothur.org/w/images/1/1a/Silva.eukarya.zip
!curl -o ../../SeqData/db/silva_A.zip http://www.mothur.org/w/images/3/3c/Silva.archaea.zip
!unzip ../../SeqData/db/silva_B.zip
!unzip ../../SeqData/db/silva_E.zip
!unzip ../../SeqData/db/silva_A.zip
# Getting the Silva databases that mothur recommends
# We aren't using these for our taxonomy assignments, per se - just to align our sequences for QC

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.3M  100 24.3M    0     0   637k      0  0:00:39  0:00:39 --:--:--  689k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2464k  100 2464k    0     0   553k      0  0:00:04  0:00:04 --:--:--  553k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3773k  100 3773k    0     0   482k      0  0:00:07  0:00:07 --:--:--  642k
Archive:  ../../SeqData/db/silva_B.zip
   creating: silva.bacteria/
  inflating: silva.bacteria/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/silva.bacteria/
  inflating: __MACOSX/silva.bacteria/._.DS_Store  
  inflating: silva.bacteria/silva.bacteria.fasta  
  inflating: __MAC

In [14]:
!cat silva.bacteria/silva.bacteria.fasta \
    silva.eukarya/silva.eukarya.fasta \
    Silva.archaea/silva.archaea.fasta \
    > ../../SeqData/db/silva.total.fasta
# Joining the ref files together

In [23]:
%%mothur
filter.seqs(vertical=t, fasta=../../SeqData/db/silva.total.fasta, processors=4)
# Just gets rid of vertical gaps in alignment in the database - will speed things up later on.

mothur > filter.seqs(vertical=t, fasta=../../SeqData/db/silva.total.fasta, processors=4)

Using 4 processors.
Creating Filter...


Running Filter...



Length of filtered alignment: 11616
Number of columns removed: 38384
Length of the original alignment: 50000
Number of sequences used to construct filter: 18491

Output File Names:
../../SeqData/db/silva.filter
../../SeqData/db/silva.total.filter.fasta


mothur > # Just gets rid of vertical gaps in alignment in the database - will speed things up later on.
[ERROR]: You are missing (
Invalid.

mothur > quit()


In [24]:
%%mothur
summary.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, processors=4)

mothur > summary.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	11562	1252	0	4	1
2.5%-tile:	1	11616	1396	0	5	463
25%-tile:	1	11616	1423	0	5	4623
Median: 	1	11616	1452	0	5	9246
75%-tile:	1	11616	1467	0	6	13869
97.5%-tile:	1	11616	1741	3	7	18029
Maximum:	1	11616	2920	5	10	18491
Mean:	1	11616	1464.96	0.351685	5.52918
# of Seqs:	18491

Output File Names:
../../SeqData/db/silva.total.filter.summary

It took 4 secs to summarize 18491 sequences.

mothur > quit()


Trimming the sequences in the database - our primers actually trim it back to about just the bacterial database (14868 seqs/18491). We might try keeping the non-primer matched sequences in there, but just using the bacterial database (14956). 

Actually, there must be some archeal or whatever in there, because trimming the bacterial db using pcr.seqs cuts it down to 13615 sequences. So, might as well go with the full, filtered db.

In [25]:
%%mothur
pcr.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, oligos=../../SeqData/primers.txt, processors=4)

mothur > pcr.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, oligos=../../SeqData/primers.txt, processors=4)

Using 4 processors.

Output File Names:
../../SeqData/db/silva.total.filter.pcr.fasta
../../SeqData/db/silva.total.filter.bad.accnos
../../SeqData/db/silva.total.filter.scrap.pcr.fasta


It took 12 secs to screen 18491 sequences.

mothur > quit()


In [26]:
%%mothur
summary.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta, processors=4)

mothur > summary.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta)

Using 1 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	4186	7013	230	0	3	1
2.5%-tile:	4187	7013	252	0	4	372
25%-tile:	4187	7013	253	0	4	3718
Median: 	4187	7013	253	0	4	7435
75%-tile:	4187	7013	253	0	5	11152
97.5%-tile:	4187	7013	254	1	6	14497
Maximum:	4187	7060	311	5	9	14868
Mean:	4187	7013	253.048	0.0471482	4.57378
# of Seqs:	14868

Output File Names:
../../SeqData/db/silva.total.filter.pcr.summary

It took 5 secs to summarize 14868 sequences.

mothur > quit()


PCRing the reference db with our sequences yielded only 250bp or so sequences, but ours are longer... I can try to expand the db by cutting at, say, 3600 and 7500 and see if that makes a difference - keeping still the smaller db for faster alignment.

It is better - come out more with 293bp seqs, not 250bp. I played around expanding or contracting it to optimize time/size.

In [160]:
%%mothur
pcr.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, start=3200, end=7200, processors=4)

mothur > pcr.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, start=3200, end=7200, processors=4)

Using 4 processors.

Output File Names:
../../SeqData/db/silva.total.filter.pcr.fasta


It took 9 secs to screen 18491 sequences.

mothur > quit()


In [161]:
%%mothur
summary.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta, processors=4)

mothur > summary.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	3201	7159	335	0	3	1
2.5%-tile:	3201	7175	354	0	4	463
25%-tile:	3201	7180	356	0	4	4623
Median: 	3201	7180	357	0	5	9246
75%-tile:	3201	7184	358	0	5	13869
97.5%-tile:	3211	7188	528	1	6	18029
Maximum:	3221	7200	1166	5	9	18491
Mean:	3202.67	7180.74	369.108	0.0754421	4.78254
# of Seqs:	18491

Output File Names:
../../SeqData/db/silva.total.filter.pcr.summary

It took 4 secs to summarize 18491 sequences.

mothur > quit()


In [231]:
%%mothur
filter.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta)

mothur > filter.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta)

Using 1 processors.
Creating Filter...


Running Filter...



Length of filtered alignment: 4000
Number of columns removed: 7616
Length of the original alignment: 11616
Number of sequences used to construct filter: 18491

Output File Names:
../../SeqData/db/silva.filter
../../SeqData/db/silva.total.filter.pcr.filter.fasta


mothur > quit()


In [4]:
!head ../../SeqData/db/silva.total.filter.pcr.fasta

>AF515816.1
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................